## Named Entity Recognition 
The chatbot should, depending on the user's intent, identify and label entities, storing them in its dialogue management system to enhance the accuracy of its responses. This notebook is dedicated to extracting entities from each Tweet, which will later be integrated into my processing pipeline.

Specifically, I want to take an utterance as input and produce an output that includes all the entities within that utterance, stored in a dictionary.

In [2]:
import sys 
print(sys.executable)

/Library/Frameworks/Python.framework/Versions/3.12/bin/python3


In [1]:
# Import libraries and set up environment
import pandas as pd
print(f"Pandas: {pd.__version__}")
import numpy as np
print(f"Numpy: {np.__version__}")

# Deep Learning 
import tensorflow as tf
print(f"Tensorflow: {tf.__version__}")
from tensorflow import keras
print(f"Keras: {keras.__version__}")
import sklearn
print(f"Sklearn: {sklearn.__version__}")

# NER
import spacy
print(f'spaCy: {spacy.__version__}')
from spacy import displacy
import random
from spacy.matcher import PhraseMatcher
import plac
from pathlib import Path

# Visualization 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

# Cool progress bars
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()  # Enable tracking of execution progress

import collections
import yaml
import pickle

# Reading back in intents
with open(r'objects/intents.yml') as file:
    intents = yaml.load(file, Loader=yaml.FullLoader)

# Reading in representative intents
# with open(r'objects/intents_repr.yml') as file:
#     intents_repr = yaml.load(file, Loader=yaml.FullLoader)

# Cool progress bars
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()  # Enable tracking of execution progress

from IPython.core.display import display, HTML
    
# Reading in training data
train = pd.read_pickle('../objects/train.pkl')

print(train.head())
print(f'\nintents:\n{intents}')

# Reading in processed data
processed = pd.read_pickle('../data/processed/processed_v2.pkl.pkl')

Pandas: 2.2.2
Numpy: 1.26.4


ModuleNotFoundError: No module named 'tensorflow'